### Things that need to be done
 
* Create the a test database that has:
  * base_data
  * accel_data
  
* Add on the following tables
  * bearing_capacity
  
* Add on the followinig tables
  * bluedrop config
  * bluedrop calibration data

* Trim the top of the bearing data plot (doesn't make a lot of sense but sure)
* 
### Options for formattig data
* Use a JSON for a quickoverview of the survey metadata
* Use a csv format to store information about each of the drops

In [1]:
# Library installs
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
# Local loads
from lib.data_classes.pffpFolder import pffpDataFolder
from lib.signal_processing.signal_function import moving_average, find_deriv_change, find_drops
from lib.general_functions.global_constants import GRAVITY_CONST
from lib.general_functions.general_function import convert_time_units


In [3]:
# Data folder
folder_dir = r"C:\Geotech_Research\Field_Analysis\BlueDrop_Analysis_Lib\stark_drops"

# PFFP sensor data
calibration_dir = r"calibration_factors\BlueDrop_Calibration_Factors.xlsx"

# PFFP tip information
tip_dir = r"C:\Geotech_Research\Field_Analysis\BlueDrop_Analysis_Lib\calibration_factors\BluedDrop_Cone_Values.xlsx"


In [4]:
# TODO: Combine the pffp_id and the read_calibration_excel_sheet just have the objet call inside
name = "foo"
id = 9
test_folder = pffpDataFolder(folder_dir, pffp_id = id, calibration_factor_dir=calibration_dir, survey_name = name)

# Read the calibration data
test_folder.read_calibration_excel_sheet()
test_folder.calibration_excel_sheet

# Select the specific calibration params
if id == 3:
    test_folder.get_sensor_calibration_params("Sequim_2021")
elif id == 1:
    test_folder.get_sensor_calibration_params("March_2023")
elif id ==9:
    test_folder.get_sensor_calibration_params("October_2023")
else:
    raise ValueError("not a valid pffp id")

In [5]:
# Init a pffpFile instances for each binary file and store the results in the folder
test_folder.store_pffp_files(recursive = False)

print(test_folder) # Information about the folder

Folder: C:\Geotech_Research\Field_Analysis\BlueDrop_Analysis_Lib\stark_drops 
Date range: Not set 
PFFP id: 9                 
Calibration Param dir: calibration_factors\BlueDrop_Calibration_Factors.xlsx 
Num .bin files: 1                 
Num files with drops: Not Set


In [6]:
# Analyze the selected pffp files
test_folder.analyze_all_files(store_df=True, use_pore_pressure=True, 
                              select_accel=["2g_accel", "50g_accel", "250g_accel"],
                              debug = False)
print(test_folder)

test_folder.process_drop_files()

# Store the first drop file
file = test_folder.pffp_drop_files[0]
drop = file.drops[0]

pffp_id = test_folder.pffp_id
drop.get_pffp_tip_values(pffp_id=pffp_id, tip_type="cone", date_string = "April_2024", file_dir = tip_dir)

drop.water_drop = True
drop.convert_tip_vals()

for name in ["projected", "mantle"]:
    drop.calc_drop_contact_area(area_type = name)
    drop.calc_drop_dynamic_bearing()
    drop.calc_drop_qs_bearing(k_factor = 0.3)

Folder 'C:\Geotech_Research\Field_Analysis\BlueDrop_Analysis_Lib\stark_drops\no_drop_folder' already exists.
Folder 'C:\Geotech_Research\Field_Analysis\BlueDrop_Analysis_Lib\stark_drops\funky' already exists.

Progress finding files with drops...
[==================================================] 100.00% ETA (min): 0.0
Initial analysis complete!
Folder: C:\Geotech_Research\Field_Analysis\BlueDrop_Analysis_Lib\stark_drops 
Date range: Not set 
PFFP id: 9                 
Calibration Param dir: calibration_factors\BlueDrop_Calibration_Factors.xlsx 
Num .bin files: 1                 
Num files with drops: 1

Progress processing drops in files...
[==================================================] 100.00% ETA (min): 1.7e-08

c:\Geotech_Research\Field_Analysis\BlueDrop_Analysis_Lib\lib\mechanics_functions\bearing_capacity_funcs.py:121: RuntimeWarning: invalid value encountered in log10
  log_val = np.log10(velocity/ref_velocity)
c:\Geotech_Research\Field_Analysis\BlueDrop_Analysis_Lib\lib\mechanics_functions\bearing_capacity_funcs.py:121: RuntimeWarning: invalid value encountered in log10
  log_val = np.log10(velocity/ref_velocity)


In [7]:
# Make the drop id
drop.make_drop_name()
drop.get_drop_datetime(file)
drop.get_peak_impulse_deceleration()

print(drop.name)

foo_0C25_index_1


In [8]:
import sqlite3

In [9]:
from BlueDrop_db_lib.dbClass import PFFPDatabase
 

In [10]:
# Input fake date information
survey_year = "24"
survey_month = "01"
survey_start_day = "01"

survey_name = test_folder.survey_name
survey_id = "{}_YR{}{}{}".format(survey_name, survey_year, survey_month, survey_start_day)

In [11]:
database = PFFPDatabase("foo2")
database.create_database()


Database 'foo2' created successfully.


In [12]:
database.add_drop_data(drop, survey_name=survey_name, survey_id = survey_id)

Accelerometer table created successfully.
Accelerometer data added successfully.
Accelerometer data added successfully.
Drop data added successfully.


In [13]:
database.view_entries("Drops")

('20240501020646021546', 'foo_0C25_index_1', '2024-04-24T20:01:01.810452', 'foo', 'foo_YR240101', '2024-05-01T02:06:46.021546')


In [14]:
accel_table_name= database.table_names["accel"]

database.view_entries(accel_table_name)

('20240501020646021546', '2024-04-24T20:01:01.810452', b'\xc04\x00\x00\x00\x00\x00\x00', b're\xc0\t\x1b\x02\x1b@W\x92v\x1c\x9e\x02\x1b@>\xbf,/!\x03\x1b@$\xec\xe2A\xa4\x03\x1b@\t\x19\x99T\'\x04\x1b@\xf0EOg\xaa\x04\x1b@\xd6r\x05z-\x05\x1b@\xbc\x9f\xbb\x8c\xb0\x05\x1b@\xa2\xccq\x9f3\x06\x1b@\x88\xf9\'\xb2\xb6\x06\x1b@n&\xde\xc49\x07\x1b@TS\x94\xd7\xbc\x07\x1b@:\x80J\xea?\x08\x1b@ \xad\x00\xfd\xc2\x08\x1b@\x06\xda\xb6\x0fF\t\x1b@\xec\x06m"\xc9\t\x1b@\xd23#5L\n\x1b@\xb8`\xd9G\xcf\n\x1b@\x9e\x8d\x8fZR\x0b\x1b@\x84\xbaEm\xd5\x0b\x1b@j\xe7\xfb\x7fX\x0c\x1b@P\x14\xb2\x92\xdb\x0c\x1b@6Ah\xa5^\r\x1b@\x1cn\x1e\xb8\xe1\r\x1b@\x02\x9b\xd4\xcad\x0e\x1b@\xe8\xc7\x8a\xdd\xe7\x0e\x1b@\xce\xf4@\xf0j\x0f\x1b@\xb4!\xf7\x02\xee\x0f\x1b@\x9aN\xad\x15q\x10\x1b@\x80{c(\xf4\x10\x1b@f\xa8\x19;w\x11\x1b@L\xd5\xcfM\xfa\x11\x1b@2\x02\x86`}\x12\x1b@\x18/<s\x00\x13\x1b@\xfe[\xf2\x85\x83\x13\x1b@\xe4\x88\xa8\x98\x06\x14\x1b@\xca\xb5^\xab\x89\x14\x1b@\xb0\xe2\x14\xbe\x0c\x15\x1b@\x96\x0f\xcb\xd0\x8f\x15\x1b@|<\x81\xe3\

In [15]:
database.close_database()

Database 'foo2' closed successfully.
